In [ ]:
import torch

import os
os.environ["LFI_PROJECT_DIR"] = '/home/ge57buc/code/lfi_mackelab/'

import sbi.inference as inference
import sbi.simulators as simulators
import sbi.utils as utils

from matplotlib import pyplot as plt

# use GPU if available
if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.set_default_tensor_type("torch.cuda.FloatTensor")
else:
    device = torch.device("cpu")
    torch.set_default_tensor_type("torch.FloatTensor")

# get simulator and prior
simulator, prior = simulators.get_simulator_and_prior("nonlinear-gaussian")

# get neural posterior (here a MAF)
neural_posterior = utils.get_neural_posterior(
    "mdn-1",
    parameter_dim=simulator.parameter_dim,
    observation_dim=simulator.observation_dim,
    simulator=simulator,
)

# create inference method
inference_method = inference.CDELFI(
    simulator=simulator,
    prior=prior,
    true_observation=simulator.get_ground_truth_observation(),
    neural_posterior=neural_posterior,
)

# run inference
inference_method.run_inference(num_rounds=4, num_simulations_per_round=500, num_components=20)

# sample posterior
samples = inference_method.sample_posterior(num_samples=10000)

# plot samples
utils.plot_hist_marginals(
    utils.tensor2numpy(samples),
    lims=simulator.parameter_plotting_limits,
    ground_truth=utils.tensor2numpy(simulator.get_ground_truth_parameters()).reshape(
        -1
    ),
)
plt.show()

In [ ]:
inference_method._neural_posterior._hidden_net

In [ ]:
inference_method._neural_posterior._hidden_net[5].weight.device

In [ ]:
simulator.get_ground_truth_observation()

In [ ]:
torch.get_

In [ ]:
logits, means, precisions, _, _ = inference_method._neural_posterior.get_mixture_components(simulator.get_ground_truth_observation())
m0, P0 = torch.zeros(5), torch.eye(5)
mp, Pp = torch.ones(5), 2*torch.eye(5)
out = inference_method._neural_posterior.posthoc_correction(logits, means, precisions, m0, P0, mp, Pp)
[out[i].shape for i in range(len(out))]

In [ ]:
logits.requires_grad, means.requires_grad, precisions.requires_grad

In [ ]:
# plot samples
samples = inference_method.sample_posterior(num_samples=10000)

utils.plot_hist_marginals(
    utils.tensor2numpy(samples),
    lims=simulator.parameter_plotting_limits,
    ground_truth=utils.tensor2numpy(simulator.get_ground_truth_parameters()).reshape(
        -1
    ),
)
plt.show()

In [ ]:
# plot samples
proposal = inference_method._model_bank[0].get_mixture_components(simulator.get_ground_truth_observation())

samples = inference_method._neural_posterior.sample(
                                           num_samples=10000,
                                           context=simulator.get_ground_truth_observation().reshape(1,-1),
                                           correction_factors={
                                               'm0' : (prior.high-prior.low)/2.,
                                               'P0' : 1e-6 * torch.eye(prior.high.shape[0]),
                                               'mp' : proposal[1].squeeze(),
                                               'Pp' : proposal[2].squeeze()
                                           }).squeeze()

utils.plot_hist_marginals(
    utils.tensor2numpy(samples),
    lims=simulator.parameter_plotting_limits,
    ground_truth=utils.tensor2numpy(simulator.get_ground_truth_parameters()).reshape(
        -1
    ),
)
plt.show()

In [ ]:
a, b  = torch.ones((1,1)), torch.zeros((1,1))
c = torch.mm(a,b)
a.requires_grad, b.requires_grad, c.requires_grad